In [1]:
from scipy import misc, ndimage, signal
from sklearn.model_selection  import train_test_split
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from time import time
import time as tm
import datetime
from operator import itemgetter
import glob
from skimage.util.shape import view_as_blocks
from keras.utils import np_utils
from keras.utils import to_categorical

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import time as tm
from sklearn.utils import resample

#Visualizers
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC
import matplotlib.pyplot as plt

#Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import matthews_corrcoef
from sklearn import model_selection

#Classifiers
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import warnings
warnings.filterwarnings('ignore')

In [2]:
!nvidia-smi
#!kill 1054324

Mon Feb 22 19:03:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:0A:00.0  On |                  N/A |
|  0%   41C    P8    15W / 250W |    371MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
path_folder = "./"
path_img_base = './images'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

In [4]:
################################################## 30 original SRM FILTERS
srm_weights = np.load(path_folder+'/SRM_Kernels1.npy') 
print (srm_weights.shape)
################################################## 30 optimized SRM FILTERS
srm_weights2 = np.load(path_folder+'/SRM_Kernels2.npy') 
print (srm_weights2.shape)
################################################## bias
biasSRM=numpy.ones(30)
################################################## TLU ACTIVATION FUNCTION
def Tanh3(x):
    tanh3 = tf.keras.activations.tanh(x)*3
    return tanh3
##################################################

(5, 5, 1, 30)
(5, 5, 1, 30)


In [5]:
def GBRAS_Net2():
    tf.keras.backend.clear_session()
    #Inputs
    inputs = tf.keras.Input(shape=(256,256,1), name="input_1")
    #Layer 1
    layers_ty = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=False, activation=Tanh3, use_bias=True)(inputs)
    layers_tn = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), padding='same', trainable=True, activation=Tanh3, use_bias=True)(inputs)
  
    layers1 = tf.keras.layers.add([layers_ty, layers_tn])
    layers1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers1)
    #Layer 2
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers1)
    layers = tf.keras.layers.SeparableConv2D(30,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 3
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers)
    layers = tf.keras.layers.SeparableConv2D(30,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    skip1 =   tf.keras.layers.Add()([layers1, layers2])
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(skip1) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 5
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers)
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 6
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 7
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 8
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers3)
    layers = tf.keras.layers.SeparableConv2D(60,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 9
    layers = tf.keras.layers.DepthwiseConv2D(1)(layers)
    layers = tf.keras.layers.SeparableConv2D(60,(3,3), padding='same', activation=None,depth_multiplier=3)(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    skip2 =   tf.keras.layers.Add()([layers3, layers4])
    #Layer 10
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(skip2) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 11
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 12
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 13
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 14
    layers = tf.keras.layers.Conv2D(60, (3,3), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 15
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    #Layer 16
    layers = tf.keras.layers.Conv2D(30, (1,1), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 17
    layers = tf.keras.layers.Conv2D(2, (1,1), strides=(1,1), activation=None, padding='same', kernel_initializer='glorot_uniform')(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    #Layer 18
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    #Layer 19
    predictions = tf.keras.layers.Softmax(axis=1)(layers)
    #Model generation
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    #Optimizer
    optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    print ("Model GBRAS-Net Generated")
    #Model compilation
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
model_untrained = GBRAS_Net2() 

Model GBRAS-Net Generated


In [7]:
path_model_trained2 = path_folder+"/GBRAS_WOW04_BIFURCATION.hdf5"
model_trained2 = tf.keras.models.load_model(path_model_trained2, custom_objects={'Tanh3':Tanh3}, compile=True)

In [8]:
path_stego = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/WOW/0.4bpp/stego"
path_cover = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/cover"

n=256
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in sorted(files):
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X

Xc_ = load_images(path_cover+'/*.pgm') ##COVER IMAGES
Xs_ = load_images(path_stego+'/*.pgm') ##STEGO IMAGES
X_  = (numpy.vstack((Xc_, Xs_)))
Xt_ = (numpy.hstack(([0]*len(Xc_), [1]*len(Xs_))))
Xt_ = np_utils.to_categorical(Xt_, 2)
X_  = np.rollaxis(X_,1,4)  #channel axis shifted to last axis

print("Total image data and labels",X_.shape,Xt_.shape)
#Cover hasta las 10000 ##Train hasta las 4000 ##Valid hasta de las 4000 a las 5000 ##Test de las 5000 a las 10000
X_train = np.concatenate([X_[0:4000],X_[10000:14000]],axis=0)
X_valid = np.concatenate([X_[4000:5000],X_[14000:15000]],axis=0)
X_test  = np.concatenate([X_[5000:10000],X_[15000:20000]],axis=0)
y_train = np.concatenate([Xt_[0:4000],Xt_[10000:14000]],axis=0)
y_valid = np.concatenate([Xt_[4000:5000],Xt_[14000:15000]],axis=0)
y_test  = np.concatenate([Xt_[5000:10000],Xt_[15000:20000]],axis=0)
#Controled randomized data for training
X_dat0, X_dat1, y_dat0, y_dat1 = train_test_split(X_train, y_train, test_size=0.50, random_state=64) 
X_train = np.concatenate([X_dat0,X_dat1],axis=0) 
y_train = np.concatenate([y_dat0,y_dat1],axis=0) 
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)


Total image data and labels (20000, 256, 256, 1) (20000, 2)
(8000, 256, 256, 1)
(8000, 2)
(2000, 256, 256, 1)
(2000, 2)
(10000, 256, 256, 1)
(10000, 2)


In [9]:
loss,accuracy = model_untrained.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_trained2.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

Loss=0.7391 and Accuracy=0.500
Loss=0.3849 and Accuracy=0.903


# ML 

In [10]:
def classifier_metrics():  
    def metrics(model):
        start_time = tm.time()
        model.fit(X_train_new, y_train_new)  
        TIME1 = tm.time() - start_time 
        print("Training Time: {0:.3f} [seconds]".format(TIME1))

        start_time = tm.time()
        y_pred = model.predict(X_test_new)
        TIME2 = tm.time() - start_time 
        print("Prediction Time: {0:.3f} [seconds]\n".format(TIME2))

        try: 
            y_prob = model.predict_proba(X_test_new)
            log_metric = log_loss(y_test_new,y_prob)
        except:
            y_prob = "Not probablistic"
            log_metric = 0 
        else:
            y_pred = model.predict(X_test_new)

        ###Set of metrics
        acc_score=accuracy_score(y_test_new,y_pred) 
        #c_k_s=cohen_kappa_score(y_test_new,y_pred)
        #zero_met=zero_one_loss(y_test_new,y_pred)
        #hl=hamming_loss(y_test_new,y_pred)
        #mc=matthews_corrcoef(y_test_new,y_pred)
        print('accuracy_score: {0:.3f}'.format(acc_score))
        #print('cohen_kappa_score: {0:.3f}'.format(c_k_s))
        #print('log_loss: {0:.3f}'.format(log_metric))
        #print('zero_one_loss: {0:.3f}'.format(zero_met))
        #print('matthews_corrcoef: {0:.3f}'.format(mc))
        
        ###Run time
        print("run_time: {0:.3f} [seconds]".format(TIME1+TIME2))
 
    for name in classifiers:
        print (str(name))
        metrics(name)
        print()
        print ("---------------------------------------------------------------------------------\n") 

In [11]:
classifiers=[
svm.SVC(gamma=0.0001,C=1.0),
LogisticRegression(),
ExtraTreesClassifier(),
RandomForestClassifier(),
LinearDiscriminantAnalysis(),
GradientBoostingClassifier(),
GaussianNB(),
KNeighborsClassifier(),
AdaBoostClassifier(),     
DecisionTreeClassifier(),
QuadraticDiscriminantAnalysis()
] 

In [12]:
def dataset_act_maps_generator(selected_layer,model):
    global X_train_new, X_valid_new, X_test_new, y_train_new, y_valid_new, y_test_new
    #X_valid
    dataset = X_valid

    image = dataset[0]
    layer_output=model.get_layer(selected_layer).output
    intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
    image=numpy.reshape(image,(1,256,256,1))
    intermediate_prediction=intermediate_model.predict(image)
    X_set = intermediate_prediction

    for i in range(len(dataset)-1):
        image = dataset[i+1]
        layer_output=model.get_layer(selected_layer).output
        intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
        image=numpy.reshape(image,(1,256,256,1))
        intermediate_prediction=intermediate_model.predict(image)
        X_set = np.concatenate((X_set,intermediate_prediction),axis=0)

    print('Data shape:',X_set.shape) 

    X_valid_new = X_set

    #X_train
    dataset = X_train
    model = model_trained2

    image = dataset[0]
    layer_output=model.get_layer(selected_layer).output
    intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
    image=numpy.reshape(image,(1,256,256,1))
    intermediate_prediction=intermediate_model.predict(image)
    X_set = intermediate_prediction

    for i in range(len(dataset)-1):
        image = dataset[i+1]
        layer_output=model.get_layer(selected_layer).output
        intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
        image=numpy.reshape(image,(1,256,256,1))
        intermediate_prediction=intermediate_model.predict(image)
        X_set = np.concatenate((X_set,intermediate_prediction),axis=0)

    print('Data shape:',X_set.shape) 

    X_train_new = X_set

    #X_test
    dataset = X_test
    model = model_trained2

    image = dataset[0]
    layer_output=model.get_layer(selected_layer).output
    intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
    image=numpy.reshape(image,(1,256,256,1))
    intermediate_prediction=intermediate_model.predict(image)
    X_set = intermediate_prediction

    for i in range(len(dataset)-1):
        image = dataset[i+1]
        layer_output=model.get_layer(selected_layer).output
        intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
        image=numpy.reshape(image,(1,256,256,1))
        intermediate_prediction=intermediate_model.predict(image)
        X_set = np.concatenate((X_set,intermediate_prediction),axis=0)

    print('Data shape:',X_set.shape) 

    X_test_new = X_set

    np.save("./Generated_datasets/"+selected_layer+"_X_valid_new",X_valid_new)#poner nombre
    np.save("./Generated_datasets/"+selected_layer+"_X_train_new",X_train_new)
    np.save("./Generated_datasets/"+selected_layer+"_X_test_new",X_test_new)

    X_valid_new = np.load("./Generated_datasets/"+selected_layer+"_X_valid_new.npy")
    X_train_new = np.load("./Generated_datasets/"+selected_layer+"_X_train_new.npy")
    X_test_new  = np.load("./Generated_datasets/"+selected_layer+"_X_test_new.npy")

    print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
    X_valid_new = np.reshape(X_valid_new, (len(X_valid_new), int(X_valid_new.shape[1]*X_valid_new.shape[2]*X_valid_new.shape[3])))
    X_train_new = np.reshape(X_train_new, (len(X_train_new), int(X_train_new.shape[1]*X_train_new.shape[2]*X_train_new.shape[3])))
    X_test_new = np.reshape(X_test_new, (len(X_test_new),    int(X_test_new.shape[1]*X_test_new.shape[2]*X_test_new.shape[3])))
    print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
    y_train_new=y_train[:,1]
    y_valid_new=y_valid[:,1]
    y_test_new =y_test[:,1]
    print(X_train_new.shape)
    print(y_train_new.shape)
    print(X_valid_new.shape)
    print(y_valid_new.shape)
    print(X_test_new.shape)
    print(y_test_new.shape)


# Generating dataset from the activation maps

In [13]:
model = model_trained2

In [14]:
for i, layer in enumerate(model.layers): 
    print(i, layer.name) 


0 input_1
1 conv2d
2 conv2d_1
3 add
4 batch_normalization
5 depthwise_conv2d
6 separable_conv2d
7 re_lu
8 batch_normalization_1
9 depthwise_conv2d_1
10 separable_conv2d_1
11 re_lu_1
12 batch_normalization_2
13 add_1
14 conv2d_2
15 re_lu_2
16 batch_normalization_3
17 conv2d_3
18 re_lu_3
19 batch_normalization_4
20 average_pooling2d
21 conv2d_4
22 re_lu_4
23 batch_normalization_5
24 depthwise_conv2d_2
25 separable_conv2d_2
26 re_lu_5
27 batch_normalization_6
28 depthwise_conv2d_3
29 separable_conv2d_3
30 re_lu_6
31 batch_normalization_7
32 add_2
33 conv2d_5
34 re_lu_7
35 batch_normalization_8
36 average_pooling2d_1
37 conv2d_6
38 re_lu_8
39 batch_normalization_9
40 average_pooling2d_2
41 conv2d_7
42 re_lu_9
43 batch_normalization_10
44 average_pooling2d_3
45 conv2d_8
46 re_lu_10
47 batch_normalization_11
48 conv2d_9
49 re_lu_11
50 batch_normalization_12
51 global_average_pooling2d
52 softmax


In [ ]:
selected_layer = 'batch_normalization_12'

#Dataset generation
dataset_act_maps_generator(selected_layer,model)

#Deploy aggregate metrics 
classifier_metrics() 

In [ ]:
selected_layer = 'conv2d_9'

#Dataset generation
dataset_act_maps_generator(selected_layer,model)

#Deploy aggregate metrics 
classifier_metrics() 

In [ ]:
selected_layer = 'batch_normalization_11'

#Dataset generation
dataset_act_maps_generator(selected_layer,model)

#Deploy aggregate metrics 
classifier_metrics() 

In [ ]:
selected_layer = 'conv2d_8'

#Dataset generation
dataset_act_maps_generator(selected_layer,model)

#Deploy aggregate metrics 
classifier_metrics() 

# Using saved data

In [15]:
selected_layer='batch_normalization_12'
X_valid_new = np.load("./Generated_datasets/"+selected_layer+"_X_valid_new.npy")
X_train_new = np.load("./Generated_datasets/"+selected_layer+"_X_train_new.npy")
X_test_new  = np.load("./Generated_datasets/"+selected_layer+"_X_test_new.npy")

print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
X_valid_new = np.reshape(X_valid_new, (len(X_valid_new), int(X_valid_new.shape[1]*X_valid_new.shape[2]*X_valid_new.shape[3])))
X_train_new = np.reshape(X_train_new, (len(X_train_new), int(X_train_new.shape[1]*X_train_new.shape[2]*X_train_new.shape[3])))
X_test_new = np.reshape(X_test_new, (len(X_test_new),    int(X_test_new.shape[1]*X_test_new.shape[2]*X_test_new.shape[3])))
print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
y_train_new=y_train[:,1]
y_valid_new=y_valid[:,1]
y_test_new =y_test[:,1]
print(X_train_new.shape)
print(y_train_new.shape)
print(X_valid_new.shape)
print(y_valid_new.shape)
print(X_test_new.shape)
print(y_test_new.shape)

#Deploy aggregate metrics 
classifier_metrics() 

(2000, 16, 16, 2) (8000, 16, 16, 2) (10000, 16, 16, 2)
(2000, 512) (8000, 512) (10000, 512)
(8000, 512)
(8000,)
(2000, 512)
(2000,)
(10000, 512)
(10000,)
SVC(gamma=0.0001)
Training Time: 8.272 [seconds]
Prediction Time: 7.735 [seconds]

accuracy_score: 0.905
run_time: 16.007 [seconds]

---------------------------------------------------------------------------------

LogisticRegression()
Training Time: 0.506 [seconds]
Prediction Time: 0.008 [seconds]

accuracy_score: 0.902
run_time: 0.514 [seconds]

---------------------------------------------------------------------------------

ExtraTreesClassifier()
Training Time: 1.874 [seconds]
Prediction Time: 0.195 [seconds]

accuracy_score: 0.901
run_time: 2.069 [seconds]

---------------------------------------------------------------------------------

RandomForestClassifier()
Training Time: 11.318 [seconds]
Prediction Time: 0.174 [seconds]

accuracy_score: 0.901
run_time: 11.492 [seconds]

---------------------------------------------------

In [16]:
selected_layer='conv2d_9'
X_valid_new = np.load("./Generated_datasets/"+selected_layer+"_X_valid_new.npy")
X_train_new = np.load("./Generated_datasets/"+selected_layer+"_X_train_new.npy")
X_test_new  = np.load("./Generated_datasets/"+selected_layer+"_X_test_new.npy")

print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
X_valid_new = np.reshape(X_valid_new, (len(X_valid_new), int(X_valid_new.shape[1]*X_valid_new.shape[2]*X_valid_new.shape[3])))
X_train_new = np.reshape(X_train_new, (len(X_train_new), int(X_train_new.shape[1]*X_train_new.shape[2]*X_train_new.shape[3])))
X_test_new = np.reshape(X_test_new, (len(X_test_new),    int(X_test_new.shape[1]*X_test_new.shape[2]*X_test_new.shape[3])))
print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
y_train_new=y_train[:,1]
y_valid_new=y_valid[:,1]
y_test_new =y_test[:,1]
print(X_train_new.shape)
print(y_train_new.shape)
print(X_valid_new.shape)
print(y_valid_new.shape)
print(X_test_new.shape)
print(y_test_new.shape)

#Deploy aggregate metrics 
classifier_metrics() 

(2000, 16, 16, 2) (8000, 16, 16, 2) (10000, 16, 16, 2)
(2000, 512) (8000, 512) (10000, 512)
(8000, 512)
(8000,)
(2000, 512)
(2000,)
(10000, 512)
(10000,)
SVC(gamma=0.0001)
Training Time: 8.520 [seconds]
Prediction Time: 7.688 [seconds]

accuracy_score: 0.905
run_time: 16.208 [seconds]

---------------------------------------------------------------------------------

LogisticRegression()
Training Time: 0.543 [seconds]
Prediction Time: 0.008 [seconds]

accuracy_score: 0.888
run_time: 0.551 [seconds]

---------------------------------------------------------------------------------

ExtraTreesClassifier()
Training Time: 1.880 [seconds]
Prediction Time: 0.195 [seconds]

accuracy_score: 0.899
run_time: 2.074 [seconds]

---------------------------------------------------------------------------------

RandomForestClassifier()
Training Time: 11.227 [seconds]
Prediction Time: 0.172 [seconds]

accuracy_score: 0.900
run_time: 11.398 [seconds]

---------------------------------------------------

In [17]:
selected_layer='batch_normalization_11'
X_valid_new = np.load("./Generated_datasets/"+selected_layer+"_X_valid_new.npy")
X_train_new = np.load("./Generated_datasets/"+selected_layer+"_X_train_new.npy")
X_test_new  = np.load("./Generated_datasets/"+selected_layer+"_X_test_new.npy")

print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
X_valid_new = np.reshape(X_valid_new, (len(X_valid_new), int(X_valid_new.shape[1]*X_valid_new.shape[2]*X_valid_new.shape[3])))
X_train_new = np.reshape(X_train_new, (len(X_train_new), int(X_train_new.shape[1]*X_train_new.shape[2]*X_train_new.shape[3])))
X_test_new = np.reshape(X_test_new, (len(X_test_new),    int(X_test_new.shape[1]*X_test_new.shape[2]*X_test_new.shape[3])))
print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
y_train_new=y_train[:,1]
y_valid_new=y_valid[:,1]
y_test_new =y_test[:,1]
print(X_train_new.shape)
print(y_train_new.shape)
print(X_valid_new.shape)
print(y_valid_new.shape)
print(X_test_new.shape)
print(y_test_new.shape)

#Deploy aggregate metrics 
classifier_metrics() 

(2000, 16, 16, 30) (8000, 16, 16, 30) (10000, 16, 16, 30)
(2000, 7680) (8000, 7680) (10000, 7680)
(8000, 7680)
(8000,)
(2000, 7680)
(2000,)
(10000, 7680)
(10000,)
SVC(gamma=0.0001)
Training Time: 107.918 [seconds]
Prediction Time: 128.384 [seconds]

accuracy_score: 0.906
run_time: 236.302 [seconds]

---------------------------------------------------------------------------------

LogisticRegression()
Training Time: 4.412 [seconds]
Prediction Time: 0.128 [seconds]

accuracy_score: 0.903
run_time: 4.540 [seconds]

---------------------------------------------------------------------------------

ExtraTreesClassifier()
Training Time: 11.167 [seconds]
Prediction Time: 0.332 [seconds]

accuracy_score: 0.900
run_time: 11.499 [seconds]

---------------------------------------------------------------------------------

RandomForestClassifier()
Training Time: 48.365 [seconds]
Prediction Time: 0.299 [seconds]

accuracy_score: 0.898
run_time: 48.665 [seconds]

-----------------------------------

In [18]:
selected_layer='conv2d_8'
X_valid_new = np.load("./Generated_datasets/"+selected_layer+"_X_valid_new.npy")
X_train_new = np.load("./Generated_datasets/"+selected_layer+"_X_train_new.npy")
X_test_new  = np.load("./Generated_datasets/"+selected_layer+"_X_test_new.npy")

print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
X_valid_new = np.reshape(X_valid_new, (len(X_valid_new), int(X_valid_new.shape[1]*X_valid_new.shape[2]*X_valid_new.shape[3])))
X_train_new = np.reshape(X_train_new, (len(X_train_new), int(X_train_new.shape[1]*X_train_new.shape[2]*X_train_new.shape[3])))
X_test_new = np.reshape(X_test_new, (len(X_test_new),    int(X_test_new.shape[1]*X_test_new.shape[2]*X_test_new.shape[3])))
print(X_valid_new.shape,X_train_new.shape,X_test_new.shape)
y_train_new=y_train[:,1]
y_valid_new=y_valid[:,1]
y_test_new =y_test[:,1]
print(X_train_new.shape)
print(y_train_new.shape)
print(X_valid_new.shape)
print(y_valid_new.shape)
print(X_test_new.shape)
print(y_test_new.shape)

#Deploy aggregate metrics 
classifier_metrics() 

(2000, 16, 16, 30) (8000, 16, 16, 30) (10000, 16, 16, 30)
(2000, 7680) (8000, 7680) (10000, 7680)
(8000, 7680)
(8000,)
(2000, 7680)
(2000,)
(10000, 7680)
(10000,)
SVC(gamma=0.0001)
Training Time: 443.277 [seconds]
Prediction Time: 437.573 [seconds]

accuracy_score: 0.840
run_time: 880.850 [seconds]

---------------------------------------------------------------------------------

LogisticRegression()
Training Time: 4.498 [seconds]
Prediction Time: 0.129 [seconds]

accuracy_score: 0.904
run_time: 4.627 [seconds]

---------------------------------------------------------------------------------

ExtraTreesClassifier()
Training Time: 10.517 [seconds]
Prediction Time: 0.359 [seconds]

accuracy_score: 0.899
run_time: 10.876 [seconds]

---------------------------------------------------------------------------------

RandomForestClassifier()
Training Time: 50.378 [seconds]
Prediction Time: 0.302 [seconds]

accuracy_score: 0.898
run_time: 50.680 [seconds]

-----------------------------------